Econometrics II PS 5
==
Maximilian Huber


You can find this code on my GitHub account: https://github.com/MaximilianJHuber/NYU
Question 2
==

$$\hat{\theta}_n = \Pi_{i=1}^n exp(x_i)$$
$$\sqrt{n}\,\left(\bar{x}_{n}-0\right)\sim N\left(0,6\right)\implies\sqrt{n}\,\left(exp(\bar{x}_{n})-1\right)\approx N\left(0,6\right)$$
Hence, $\mathbb{V}\left[ exp(\bar{x}_n) \right] \approx 6/n$.

In [1]:
@everywhere using Distributions
@everywhere using StatsBase

In [2]:
@everywhere function bootstrap(x,θhat,n)
    x_star = x[ceil(Integer, rand(Uniform(),n)*n)]
    θ_star = exp(mean(x_star))
    return abs(sqrt(n)*(θ_star - θhat)/sqrt(6/n))
end

In [3]:
function sim(B=10000)
    n = 10
    result = SharedArray(Bool,B,2)
    @sync @parallel for j in 1:B
        x = rand(Normal(0,6),n)
        θhat = exp(mean(x))
        
        t = Array(Float64,10000)
        for i=1:10000
            t[i] = bootstrap(x,θhat,n)
        end
        z_star = percentile(t,100-5)
        result[j,:] = [convert(Bool,sum(1 .<= θhat + sqrt(6/n) * [-z_star, z_star])%2), 
            convert(Bool,sum(1 .<= θhat + sqrt(6/n) * [-1.96, 1.96])%2)]
    end
    return result
end

sim (generic function with 2 methods)

In [4]:
@time result = sim()
sum(result,1)./size(result)[1]

 62.636689 seconds (4.01 M allocations: 175.962 MB, 0.30% gc time)


1×2 Array{Float64,2}:
 0.9735  0.6861

The bootstrap (first number) achieves much more accurate coverage than the asymptotic confidence interval.